## Imports

In [12]:
import pandas as pd
import numpy as np
import re
import os
import folium

## Data Read-in

In [36]:
df = pd.read_csv('redfin_2023-06-07-02-46-31.csv')

## Data Clean

In [37]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [38]:
df = df.dropna(subset=['SOLD DATE'])

In [39]:
# Define list of desired months (excluding current month)
desired_months = ['May']

# Filter DataFrame to include only entries from desired months
df_filtered = df[df['SOLD DATE'].str.split('-', expand=True)[0].isin(desired_months)]

# Reset the index
df_filtered = df_filtered.reset_index(drop=True)

In [40]:
# Data checks
print(df_filtered['PRICE'].isna().value_counts())
print('-------')
print(df_filtered['$/SQUARE FEET'].isna().value_counts())
print('-------')
print(df_filtered['YEAR BUILT'].isna().value_counts())
print('-------')

PRICE
False    1162
Name: count, dtype: int64
-------
$/SQUARE FEET
False    1148
True       14
Name: count, dtype: int64
-------
YEAR BUILT
False    1162
Name: count, dtype: int64
-------


In [41]:
df_filtered['PRICE'] = pd.to_numeric(df_filtered['PRICE'])
df_filtered['$/SQUARE FEET'] = pd.to_numeric(df_filtered['$/SQUARE FEET'])
df_filtered['YEAR BUILT'] = pd.to_numeric(df_filtered['YEAR BUILT'])
df_filtered['LATITUDE'] = pd.to_numeric(df_filtered['LATITUDE'])
df_filtered['LONGITUDE'] = pd.to_numeric(df_filtered['LONGITUDE'])

In [42]:
df_filtered = df_filtered.sort_values(by='PRICE',ascending=True).reset_index(inplace=False)
df_filtered = df_filtered.drop(columns='index')

In [43]:
df_filtered

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
0,PAST SALE,May-31-2023,Condo/Co-op,2030 S Ocean Dr #710,Hallandale Beach,FL,33009.0,440.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/203...,MARMLS,A11349064,N,Y,25.978036,-80.119391
1,PAST SALE,May-11-2023,Condo/Co-op,1061 45 St #8,Deerfield Beach,FL,33064.0,18000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Deerfield-Beach/1061...,Beaches MLS,F10367385,N,Y,26.285551,-80.138541
2,PAST SALE,May-24-2023,Condo/Co-op,2851 S Palm Aire Dr #209,Pompano Beach,FL,33069.0,21200.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/2851-S...,MARMLS,A11375423,N,Y,26.216672,-80.161742
3,PAST SALE,May-19-2023,Condo/Co-op,9041 Sunrise Lakes Blvd #211,Sunrise,FL,33322.0,65000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunrise/9041-Sunrise...,Beaches MLS,F10371722,N,Y,26.163517,-80.272356
4,PAST SALE,May-5-2023,Condo/Co-op,2271 NW 47th Ter #214,Lauderhill,FL,33313.0,70000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderhill/2271-NW-4...,Beaches MLS,F10370572,N,Y,26.155254,-80.213700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1157,PAST SALE,May-26-2023,Condo/Co-op,2200 N Ocean Blvd Unit N702,Fort Lauderdale,FL,33305.0,3850000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2200...,Beaches MLS,F10374536,N,Y,26.157109,-80.100473
1158,PAST SALE,May-19-2023,Condo/Co-op,2000 S Ocean Dr Unit 21B,Hallandale Beach,FL,33009.0,4000000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/200...,MARMLS,A11357515,N,Y,25.978824,-80.119610
1159,PAST SALE,May-23-2023,Condo/Co-op,701 N Fort Lauderdale Beach Blvd #702,Fort Lauderdale,FL,33304.0,4200000.0,4.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/701-...,Beaches MLS,F10358526,N,Y,26.132895,-80.104208
1160,PAST SALE,May-18-2023,Condo/Co-op,100 Las Olas Blvd Ph 4603,Fort Lauderdale,FL,33301.0,5000000.0,5.0,6.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/100-...,Beaches MLS,F10360678,N,Y,26.118855,-80.141485


In [48]:
print(df_filtered['URL'].iloc[4])

https://www.redfin.com/FL/Lauderhill/2271-NW-47th-Ter-33313/unit-214/home/41594768


In [49]:
# Correct the prices, if needed
df_filtered.at[0,'PRICE']=(440000)
df_filtered.at[1,'PRICE']=(180000)
df_filtered.at[2,'PRICE']=(212000)
df_filtered.at[3,'PRICE']=(650000)

In [50]:
# Find problem psf by searching for a '0' value
df_filtered.loc[df_filtered['$/SQUARE FEET'] == '0'][['SOLD DATE','ADDRESS','CITY','$/SQUARE FEET','PRICE','SQUARE FEET']]

,SOLD DATE,ADDRESS,CITY,$/SQUARE FEET,PRICE,SQUARE FEET


In [51]:
# Corrections, if needed
df_filtered.at[0,'$/SQUARE FEET']=(440000/1248)
# df_filtered.at[1,'$/SQUARE FEET']=(180000/1075)
df_filtered.at[2,'$/SQUARE FEET']=(212000/1120)
df_filtered.at[3,'$/SQUARE FEET']=(650000/620)

In [52]:
# Find problem psf by searching for low values
df_filtered.sort_values(by='$/SQUARE FEET',ascending=True).head(20)[['PRICE','ADDRESS','CITY','$/SQUARE FEET']]

,PRICE,ADDRESS,CITY,$/SQUARE FEET
4,70000.0,2271 NW 47th Ter #214,Lauderhill,76.0
11,80000.0,3506 NW 49th Ave #415,Lauderdale Lakes,85.0
12,81500.0,1380 NW 43rd Ter #309,Lauderhill,88.0
22,90000.0,5985 Del Lago Cir #312,Sunrise,91.0
5,70000.0,1280 NW 43rd Ave #204,Lauderhill,96.0
125,132000.0,4960 E Sabal Palm Blvd #311,Tamarac,98.0
6,71250.0,3511 Inverrary Dr #205,Lauderhill,98.0
19,88000.0,8100 SW 24th St #109,North Lauderdale,101.0
29,95000.0,4851 NW 21st St #510,Lauderhill,102.0
34,96000.0,2606 Nassau Bnd Unit B2,Coconut Creek,102.0


In [53]:
print(df_filtered.URL.iloc[4])

https://www.redfin.com/FL/Lauderhill/2271-NW-47th-Ter-33313/unit-214/home/41594768


In [79]:
# # Drop sales that aren't condos but labeled as such
# df_filtered = df_filtered.drop(1320)

## Make Maps

In [55]:
### Create a price column formatted as currency ###
df_filtered['PRICE_AS_CURRENCY'] = df_filtered['PRICE'].apply(lambda x: "${:,.0f}".format(x))
### Set formatting for Beds, Baths ###
df_filtered['YEAR BUILT DISPLAY'] = df_filtered['YEAR BUILT'].apply(lambda x: '{:.0f}'.format(x))
df_filtered['PRICE_SQUARE_FEET_AS_CURRENCY'] = df_filtered['$/SQUARE FEET'].apply(lambda x: '${:,.0f}'.format(x))

In [56]:
df_filtered = df_filtered.sort_values(by=['PRICE'], ascending=False)
### Insert different colors for top 10 sales vs. the rest ###
df_filtered['COLOR'] = ''
### Create RANK column ###
df_filtered['RANK'] = 0
### Insert RANK values ###
df_filtered['RANK'] = range(1, len(df_filtered) + 1)
# use numpy to assign values to the 'COLOR' column
df_filtered['COLOR'] = np.where(df_filtered['RANK'] <= 10, 'orange', 'blue')

## HTML Popup Formatter

In [57]:
### Define list of columns to drop from DF ###
columns_drop = ['SALE TYPE','PROPERTY TYPE','STATE OR PROVINCE','ZIP OR POSTAL CODE','HOA/MONTH','STATUS','NEXT OPEN HOUSE START TIME','NEXT OPEN HOUSE END TIME','SOURCE','MLS#','FAVORITE','INTERESTED','SQUARE FEET','LOT SIZE']

In [58]:
### Drop the columns ###
df_filtered = df_filtered.drop(columns=columns_drop)

In [59]:
def popup_html(row):
    Price = row['PRICE_AS_CURRENCY']
    Address = row['ADDRESS']
    City = row['CITY']
    sold_date = row['SOLD DATE']
    beds = row['BEDS']
    baths = row['BATHS']
    psf = row['PRICE_SQUARE_FEET_AS_CURRENCY']
    year_built = row['YEAR BUILT DISPLAY']
    rank = row['RANK']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Price: </strong>{}'''.format(Price) + '''<br>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>City: </strong>{}'''.format(City) + '''<br>
    <strong>Sold: </strong>{}'''.format(sold_date) + '''<br>
    <strong>Beds: </strong>{}'''.format(beds) + '''<br>
    <strong>Baths: </strong>{}'''.format(baths) + '''<br>
    <strong>Price per sf: </strong>{}'''.format(psf) + '''<br>
    <strong>Year Built: </strong>{}'''.format(year_built) + '''<br>
    <strong>Price Rank: </strong>{}'''.format(rank) + '''
    </html>
    '''
    return html

In [60]:
### Create map container ###
m = folium.Map(location=df_filtered[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=10,tiles=None)

### Create title ###
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"May 2023 Condo Sales")

m.get_root().html.add_child(folium.Element(title_html))

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top 10 Sales')

for index, row in df_filtered.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)

# Display map

In [61]:
m.save('index.html')

## Summary Info

In [62]:
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

Newest = '\033[1m' + 'Newest' + '\033[0m'
Oldest = '\033[1m' + 'Oldest' + '\033[0m'

In [63]:
df_filtered.columns

Index(['SOLD DATE', 'ADDRESS', 'CITY', 'PRICE', 'BEDS', 'BATHS', 'LOCATION',
       'YEAR BUILT', 'DAYS ON MARKET', '$/SQUARE FEET', 'URL', 'LATITUDE',
       'LONGITUDE', 'PRICE_AS_CURRENCY', 'YEAR BUILT DISPLAY',
       'PRICE_SQUARE_FEET_AS_CURRENCY', 'COLOR', 'RANK'],
      dtype='object')

In [64]:
df_filtered['FULL_ADDRESS'] = df_filtered['ADDRESS'] + ' ' + df_filtered['CITY']

In [74]:
print(df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['URL'])

https://www.redfin.com/FL/Hillsboro-Beach/1187-Hillsboro-Mile-33062/unit-12E/home/41516406


In [66]:
print(f"{ME}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{LE}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{MAX_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{MIN_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{Newest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{Oldest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['YEAR BUILT']:.0f}")

Most Expensive
Four Seasons, 525 N Ft Lauderdale Bch Blvd #2102 Fort Lauderdale | Price $13,250,000 | $2,133 psf | Year built: 2022
Least Expensive
PARK SOUTH TEN INC CONDO, 1280 NW 43rd Ave #204 Lauderhill | Price $70,000 | $96 psf | Year built: 1970
Highest Price Per Square Foot
Four Seasons, 525 N Ft Lauderdale Bch Blvd #2102 Fort Lauderdale | Price $13,250,000 | $2,133 psf | Year built: 2022
Lowest Price Per Square Foot
Castle Gardens, 2271 NW 47th Ter #214 Lauderhill | Price $70,000 | $76 psf | Year built: 1972
Newest
Four Seasons, 525 N Ft Lauderdale Bch Blvd #2102 Fort Lauderdale | Price $13,250,000 | $2,133 psf | Year built: 2022
Oldest
Virginia Kaye -east, 1187 Hillsboro Mile Unit 12E Hillsboro Beach | Price $500,000 | $980 psf | Year built: 1950


## Time on Market Calculator

In [92]:
# print(df_Current.loc[df_Current['YEAR BUILT'].idxmin()])

In [76]:
from datetime import datetime, timedelta

date1 = datetime(2022, 3, 6) ## List (Earlier) date
date2 = datetime(2023, 5, 9) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

429


## Map URL Snagger

In [67]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [69]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/Broward_condo_sales_month_ending_may_2023


## Get Summary Data

In [70]:
print('SALES INFO')
print(f'Number of sales: {len(df_filtered)}')
print('--------')
print(f'Total sale price: ${df_filtered["PRICE"].sum():,.0f}')
print('--------')
print(f'Median sale price: ${df_filtered["PRICE"].median():,.0f}')
print('--------')
print(f'Max sale price: ${df_filtered["PRICE"].max():,.0f}')
print('--------')
print(f'Min sale price: ${df_filtered["PRICE"].min():,.0f}')
print('------------------------------------------------')
print('PSF INFO')
print(f'Max price per square foot: ${df_filtered["$/SQUARE FEET"].max():,.0f}')
print('--------')
print(f'Min price per square foot: ${df_filtered["$/SQUARE FEET"].min():,.0f}')
print('--------')
print(f'Median price per square foot: ${df_filtered["$/SQUARE FEET"].median():,.0f}')
print('------------------------------------------------')
print('CONDO AGES')
print(f'Newest building: {df_filtered["YEAR BUILT"].max()}')
print('----------')
print(f'Oldest building: {df_filtered["YEAR BUILT"].min()}')
print('----------')
print(f'Average building age: {df_filtered["YEAR BUILT"].mean()}')

SALES INFO
Number of sales: 1162
--------
Total sale price: $413,857,616
--------
Median sale price: $240,000
--------
Max sale price: $13,250,000
--------
Min sale price: $70,000
------------------------------------------------
PSF INFO
Max price per square foot: $2,133
--------
Min price per square foot: $76
--------
Median price per square foot: $234
------------------------------------------------
CONDO AGES
Newest building: 2022.0
----------
Oldest building: 1950.0
----------
Average building age: 1981.4139414802066
